# Regex and Prodigy (Merging all enitity before correction)



<font size= "3">
Annotation of entities for sentences based on prodigy's format for different labels for further usage in Machine learning. Prodigy is an annotation tool based on spaCy. Please see here:
    </p>
  <a href="https://prodi.gy/ "> Prodigy </a> 
<font 3>

## Imports

In [11]:
import pandas as pd
import numpy as np
import re
import pandas as pd
import matplotlib.pyplot as plt
import scipy as sp
import spacy 
import re
from prodigy.util import write_jsonl
import prodigy



## Read the Data

In [12]:
importVersion = '013'

In [13]:
path= '../data/01_df_v{0}.pickle'.format(importVersion)# Put the path of the data in your local machine here, consider the letter "r" before the path
dfAstroNova = pd.read_pickle(path)

In [14]:
# Sort the data based on the chapters of the book 
dfAstroNova['chapter'] = dfAstroNova.chapter.replace("appendix b",np.nan).astype(float)  
dfAstroNova = dfAstroNova.rename_axis('MyIdx').sort_values(by = ['chapter', 'MyIdx'], ascending = [True, True])
dfAstroNova.chapter.fillna('appendix b', inplace=True)

In [15]:
dfAstroNova.reset_index(inplace=True)
dfAstroNova=dfAstroNova.drop("MyIdx",axis=1,inplace=False)
dfAstroNova=dfAstroNova.drop("html",axis=1)

In [16]:
type(dfAstroNova)

pandas.core.frame.DataFrame

In [17]:
dfAstroNova.head(5)

,text,links,italic,chapter,graphic,table,marginal,sentences,tagged
0,Chapter 1,[],[],1,[],[],[],[Chapter 1],"[[(Chapter, None), (1, NUM)]]"
1,On the distinction between the first motion an...,[],[],1,[],[],[],[On the distinction between the first motion a...,"[[(On, None), (the, None), (distinction, None)..."
2,The testimony of the ages confirms that the mo...,[],[],1,[],[],[ Terms: 1. The first motion is that of the wh...,[The testimony of the ages confirms that the m...,"[[(The, None), (testimony, None), (of, None), ..."
3,It is just this from which astronomy arose amo...,[],[],1,[ ch 1 gr 1],[],[],[It is just this from which astronomy arose am...,"[[(It, None), (is, None), (just, None), (this,..."
4,Before the distinction between the first motio...,[],[(such],1,[],[],[ 2],[Before the distinction between the first moti...,"[[(Before, None), (the, None), (distinction, N..."


### Read the Sentences

In [8]:
df=dfAstroNova.sentences

In [20]:
sents=[]
for para in dfAstroNova.sentences:
    sents +=para

# Merging  Based on Consistant format with Prodigy 

<fig size= 3>
We annotated the text label by label using regex, In this way we  could see influence of each label.Then, each annotation could be corrected by a annotater using prodigy interface. Morever we run a ner.batch-train to  use Prodigy to make more corrections. Now we can merge all versions of our data in a single version of corrected annotated data by annotater and prodigy.

    
<fig size>

## Merging DATE, TIME and ASTR

<fig size= 3>
Annotations refer to:
<ul>
    
<li>ASTR: Related phrases </li>
<li>DATE: All types of date </li>
<li>TIME: All types of time </li>
<li>PARA: Astronomical sizes</li>
<li>LONG: Related phrases to longituide </li>
<li>STAR: Related phrases to stars </li>
<li>PLAN: Related phrases to planets </li>
<li>NAMES: important names</li>
<fig size>
    </ul>

In [29]:
from prodigy.components.db import connect
from prodigy.models.ner import merge_spans

db = connect()  # connect to the DB using the prodigy.json settings
datasets = ['ner_date_v02','ner_time_v02','ner_para_v02','ner_astr_v02','ner_long_v02','ner_star_v02','ner_plan_v02','ner_name_v02']
examples = []
for dataset in datasets:
    examples += db.get_dataset(dataset)  # get examples from the database

merged_examples = merge_spans(examples)

In [30]:
from prodigy import set_hashes

merged_examples = [set_hashes(eg, overwrite=True) for eg in merged_examples]
db.add_dataset('data_merged_v02')
db.add_examples(merged_examples, datasets=['data_merged_v02'])

## See Misaligned Tokens

<fig size= 3>
This step is very important, since if Pordigy faces with Misaligned tokens, we can see that before and try to adjust regex accordingly
<fig size>

In [31]:
def misaligned_token(examples):
    counter=0
    nlp = spacy.load("en_core_web_sm")  
    for example in examples:  
        doc = nlp(example["text"])
        for span in example["spans"]:
            char_span = doc.char_span(span["start"], span["end"])
            if char_span is None:  
                counter+=1
                print("{}- Misaligned tokens-->".format(counter), example["text"], span)

In [32]:
misaligned_token(merged_examples)